<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2023-02-23 16:18:11--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G   532KB/s    in 45m 32s 

2023-02-23 17:03:44 (520 KB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
print(wv_embeddings)

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [ ]:
wv_embeddings[word]

array([ 0.6851772 , -1.2778991 , -0.41913974,  1.3623164 , -3.1675398 ,
        0.09950767,  0.6402681 , -1.1245339 , -0.6699619 , -0.6998852 ,
        0.4936771 , -0.40500194, -3.0706816 , -2.2809966 ,  0.85798043,
        2.7093108 ,  0.3492745 , -0.03494101, -0.22330493,  1.2290467 ,
        1.7755157 , -3.158358  , -0.6515983 ,  0.7224096 ,  2.3193083 ,
       -1.7969862 ,  0.40903398, -2.744604  , -1.7179952 , -0.914309  ,
       -0.75887376, -0.35140672, -0.5182776 , -1.9097351 , -0.8300773 ,
        0.02147918,  1.1783471 ,  0.03169126, -0.3069023 ,  1.6666299 ,
        0.6711357 , -2.1706133 , -0.11800487,  0.22336982, -1.2075394 ,
       -0.86297905, -0.63865614,  1.1733794 ,  0.10022762,  0.7017279 ,
        2.7290728 , -0.4640484 , -2.1719306 , -0.3562852 , -1.8449957 ,
        0.10270727,  1.1125596 , -0.8364318 ,  1.9513408 , -0.97937447,
        1.2650859 ,  0.06809282,  0.6477318 , -0.52431005, -0.6103959 ,
       -2.979829  , -0.7889965 , -0.11004248,  1.7603841 , -1.05

In [ ]:
print(f"Num of words: {len(wv_embeddings.index2word)}")

Num of words: 1787145


In [ ]:
wv_embeddings.index2word[22]

'work'

Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [ ]:
# method most_simmilar
res_dog = wv_embeddings.most_similar(positive = [word], topn = 10)
for i in range(10):
  most_similar_key, similarity = res_dog[i]
  print(f'{most_similar_key}: {similarity:.4f}')


'''your code'''

animal: 0.8564
dogs: 0.7881
mammal: 0.7624
cats: 0.7621
animals: 0.7608
feline: 0.7392
bird: 0.7315
animal1: 0.7219
doggy: 0.7213
labrador: 0.7209


'your code'

In [ ]:
wv_embeddings.distance(word, 'cat')

0.3147658705711365

In [ ]:
wv_embeddings.vectors.shape

(1787145, 200)

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [ ]:
token_prob = MyTokenizer()
for word in token_prob.tokenize("I'm telling you, Bro!".lower()):
  print(word)

i
m
telling
you
bro


In [ ]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
  features = np.zeros([dim], dtype='float64')
  n = 0
  for word in tokenizer.tokenize(question.lower()):
    if word in embeddings:
      n += 1
      features += embeddings[f'{word}']
  if n:
    return features / n
  return features

"""
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
"""


'''your code'''

'your code'

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
my_tokenizer = MyTokenizer()
text = 'I love neural networks'
feat = question_to_vec(text, wv_embeddings, my_tokenizer, 200)
print(f'{feat[2]:.2f}')

'''your code'''

-1.29


'your code'

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

Максимум равен 1


<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

DCG@10 = 0.3

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
def hits_count(dup_ranks, k):
  hits_value = sum([rank <= k for rank in dup_ranks]) / len(dup_ranks)
  return hits_value

  """
     dup_ranks: list индексов дубликатов
           result: вернуть  Hits@k
  """

  '''your code'''


In [ ]:
from math import log2

def dcg_score(dup_ranks, k):
  dcg_value = sum([(rank <= k) / (log2(1 + rank)) for rank in dup_ranks]) / len(dup_ranks)
  return dcg_value

  """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
  """
  '''your code'''


Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]

# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [candidates_ranking[0].index(copy_answers[0]) + 1]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/stackoverflow_similar_questions.zip

Archive:  /content/drive/MyDrive/stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


In [ ]:
ls -la

total 1419864
drwxr-xr-x 1 root root       4096 Feb 23 17:04  ./
drwxr-xr-x 1 root root       4096 Feb 23 16:13  ../
drwxr-xr-x 4 root root       4096 Feb 21 14:37  .config/
drwxr-xr-x 2 root root       4096 Oct 26  2018  data/
drwx------ 5 root root       4096 Feb 23 17:04  drive/
drwxrwxr-x 3 root root       4096 Sep 22  2019  __MACOSX/
drwxr-xr-x 1 root root       4096 Feb 21 14:37  sample_data/
-rw-r--r-- 1 root root 1453905423 Feb 13 01:44 'SO_vectors_200.bin?download=1'


Считайте данные.

In [ ]:
def read_corpus(filename):
   data = []
   for line in open(filename, encoding='utf-8'):
     data.append(line.strip())
   return data

Нам понадобиться только файл validation.

In [ ]:
validation_data = read_corpus('./data/validation.tsv')

In [ ]:
len(validation_data[0])

54516

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 54516
2 52817
3 54463
4 53723
5 52296


In [ ]:
validation_data[0][-1]

's'

In [ ]:
for line in validation_data:
  q, *ex = line
  print(q, line, sep='\n')
  break

H
How to print a binary heap tree without recursion?	How do you best convert a recursive function to an iterative one?	How can i use ng-model with directive in angular js	flash: drawing and erasing	toggle react component using hide show classname	Use a usercontrol from another project to current webpage	~ Paths resolved differently after upgrading to ASP.NET 4	Materialize datepicker - Rendering when an icon is clicked	Creating PyPi package - Could not find a version that satisfies the requirement iso8601	How can I analyze a confusion matrix?	How do I declare a C array in Swift?	Using rand() when flipping a coin and rolling a die	Handling a JSON field with a special character in its name in Java	React Native select row on ListView when push it	Get 'creation_time' of video using ffmpeg and regex	Does row exist and multiple where	How to specify a classifier in a gradle dependency's dependency?	Using $unwind on multiple documents	Visual Studio Code: Missing X509Certificate2UI	Developing Co

In [ ]:
validation_data[0].split('\t')

['How to print a binary heap tree without recursion?',
 'How do you best convert a recursive function to an iterative one?',
 'How can i use ng-model with directive in angular js',
 'flash: drawing and erasing',
 'toggle react component using hide show classname',
 'Use a usercontrol from another project to current webpage',
 '~ Paths resolved differently after upgrading to ASP.NET 4',
 'Materialize datepicker - Rendering when an icon is clicked',
 'Creating PyPi package - Could not find a version that satisfies the requirement iso8601',
 'How can I analyze a confusion matrix?',
 'How do I declare a C array in Swift?',
 'Using rand() when flipping a coin and rolling a die',
 'Handling a JSON field with a special character in its name in Java',
 'React Native select row on ListView when push it',
 "Get 'creation_time' of video using ffmpeg and regex",
 'Does row exist and multiple where',
 "How to specify a classifier in a gradle dependency's dependency?",
 'Using $unwind on multiple do

### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
  vect_question = question_to_vec(question, embeddings, tokenizer, dim).reshape(1, -1)
  candidates_copy = deepcopy(candidates)
#  print('vect_q.shape:', vect_question.shape)
  vect_candidates = [question_to_vec(candidate, embeddings, tokenizer, dim) for candidate in candidates_copy]
#  print('vect_candidates.shape:', len(vect_candidates), 'x', len(vect_candidates[0]))
  cos_sim = cosine_similarity(vect_question, vect_candidates).tolist()[0]
#  print('cos_sim.shape:', len(cos_sim))
#  print('cos_sim[0]:', cos_sim[0])
  result = []
  for i in range(len(candidates)):
    idx = cos_sim.index(max(cos_sim))
#    print(i, ':', idx, cos_sim)
    result.append((idx, candidates[idx]))
    cos_sim[idx] = -2
  return result

  """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
  """
  '''your code'''

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
len(candidates[0])

3

In [ ]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(0, 'Getting all list items of an unordered list in PHP'), #скрыт
            (2, 'select2 not displaying search results'), #скрыт
            (1, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

021

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
   # q, *ex = line
    line = line.split('\t')
    q = line[0]
    ex = line[1:]
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.415 | Hits@   1: 0.415
DCG@   5: 0.502 | Hits@   5: 0.582
DCG@  10: 0.525 | Hits@  10: 0.651
DCG@ 100: 0.570 | Hits@ 100: 0.874
DCG@ 500: 0.583 | Hits@ 500: 0.973
DCG@1000: 0.586 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [ ]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
'''your code'''
words = [my_tokenizer.tokenize(request) for request in train_data]

In [ ]:
len(words)

1000000

In [ ]:
for i in range(0, 5):
  print(i + 1, len(words[i]))

1 13
2 20
3 15
4 14
5 20


In [ ]:
words[0]

['converting',
 'string',
 'to',
 'list',
 'Convert',
 'Google',
 'results',
 'object',
 'pure',
 'js',
 'to',
 'Python',
 'object']

In [ ]:
for i in range(0, 5):
  print(i + 1, len(train_data[i]))

1 82
2 120
3 79
4 91
5 107


In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=5).wv

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    line = line.split('\t')
    q = line[0]
    ex = line[1:]
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.259 | Hits@   1: 0.259
DCG@   5: 0.331 | Hits@   5: 0.395
DCG@  10: 0.353 | Hits@  10: 0.464
DCG@ 100: 0.407 | Hits@ 100: 0.728
DCG@ 500: 0.431 | Hits@ 500: 0.916
DCG@1000: 0.440 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


После нормализации слов ничего не поменялось